## 1. Installion and SetUp

In [ ]:
!pip install -q transformers huggingface_hub

In [ ]:
from google.colab import userdata
import pickle
HF_TOKEN = userdata.get("HF_TOKEN")
print("Token loaded:", HF_TOKEN is not None)

Token loaded: True


## 2. Initalizing Model

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    token=HF_TOKEN
)

## 3. Add System Prompt

In [ ]:
SYSTEM_PROMPT = """
You are a helpful and friendly medical assistant.
You only answer general health-related questions.

Rules:
- Do NOT provide medical diagnosis.
- Do NOT prescribe medicines or dosages.
- Avoid harmful or emergency medical advice.
- If the question is serious, politely advise seeing a doctor.
- Keep responses clear, calm, and friendly.
"""

## 4. Utils Functions

In [ ]:
def is_unsafe_question(text):
    blocked_words = [
        "dosage", "dose", "prescription",
        "how much medicine",
        "treatment for cancer",
        "suicide", "self harm"
    ]
    return any(word in text.lower() for word in blocked_words)


In [ ]:
def health_chatbot_stream(user_question):
    if is_unsafe_question(user_question):
        print("⚠️ I can’t provide medical treatment or prescriptions. Please consult a qualified healthcare professional.")
        return

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_question}
    ]

    stream = client.chat_completion(
        messages=messages,
        max_tokens=200,
        temperature=0.6,
        stream=True
    )

    print("🤖 Chatbot:", end=" ", flush=True)

    for chunk in stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)

    print("\n")


## 5. Test

In [ ]:
health_chatbot_stream("What causes a sore throat?")


🤖 Chatbot:  A sore throat can be caused by various reasons, such as viruses, bacteria, allergies, or environmental irritants. Common viruses that cause sore throats include the flu, common cold, and strep throat. Bacterial infections, like strep throat, may require medical treatment. If you have persistent symptoms or severe pain, it's essential to consult your healthcare provider for an accurate diagnosis and appropriate treatment. In the meantime, you can try soothing your throat with warm liquids, gargling with salt water, and avoiding harsh foods or irritants.



## 6. Export as .pkl for production

In [ ]:
prompt_bundle = {
    "system_prompt": SYSTEM_PROMPT,
    "safety_function": is_unsafe_question
}

# Export as pkl
with open("health_prompt.pkl", "wb") as f:
    pickle.dump(prompt_bundle, f)